In [1]:
import pandas as pd
import pywt;

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Data Preprocessing

- Create separate metadata dataframe containing information about each case.<br>
- Create df containing all measurements for each case.

In [2]:
data = pd.read_csv("mill_data/mill_data.csv")
metadata_df = data.iloc[:167, :8]
df = data.iloc[:9000, 8:]
df.head()

,smcAC_case001,smcDC_case001,vib_table_case001,vib_spindle_case001,AE_table_case001,AE_spindle_case001,smcAC_case002,smcDC_case002,vib_table_case002,vib_spindle_case002,...,AE_table_case164,AE_spindle_case164,smcAC_case165,smcDC_case165,vib_table_case165,vib_spindle_case165,AE_table_case165,AE_spindle_case165,smcAC_case166,smcDC_case166
0,-0.017090,0.625000,0.078125,0.314941,0.087280,0.103760,0.307617,0.668945,0.075684,0.301514,...,0.114746,0.139771,0.244141,1.328125,0.063477,0.290527,0.101318,0.117798,-0.205078,1.381836
1,0.263672,0.810547,0.085449,0.301514,0.098267,0.123291,0.312500,0.678711,0.080566,0.308838,...,0.095215,0.112305,0.244141,1.333008,0.058594,0.279541,0.121460,0.140381,-0.239258,1.386719
2,0.207520,0.781250,0.078125,0.303955,0.092163,0.104980,0.195312,0.859375,0.078125,0.299072,...,0.101929,0.120239,0.205078,1.333008,0.058594,0.285645,0.122070,0.142822,-0.156250,1.386719
3,0.302734,0.849609,0.073242,0.300293,0.095215,0.111084,0.187988,1.123047,0.080566,0.306396,...,0.098877,0.118408,0.156250,1.333008,0.063477,0.286865,0.119019,0.139160,0.126953,1.381836
4,0.239258,1.098633,0.083008,0.299072,0.083008,0.092163,-0.009766,1.166992,0.075684,0.319824,...,0.098877,0.120239,0.048828,1.337891,0.058594,0.286865,0.114136,0.132446,0.200195,1.376953


Separate audio features from the rest of that data. Our analysis and modeling with focus on the audio features <br>
* AE_table: Acoustic emission at table
* AE_spindle: Acoustic emission at spindle.

In [3]:
audio_features = df[df.columns[df.columns.str.contains("AE")]]
audio_features.head()

,AE_table_case001,AE_spindle_case001,AE_table_case002,AE_spindle_case002,AE_table_case003,AE_spindle_case003,AE_table_case004,AE_spindle_case004,AE_table_case005,AE_spindle_case005,...,AE_table_case161,AE_spindle_case161,AE_table_case162,AE_spindle_case162,AE_table_case163,AE_spindle_case163,AE_table_case164,AE_spindle_case164,AE_table_case165,AE_spindle_case165
0,0.087280,0.103760,0.086670,0.099487,0.092773,0.104980,0.112915,0.139771,0.095825,0.110474,...,0.102539,0.123291,0.091553,0.098877,0.093994,0.109253,0.114746,0.139771,0.101318,0.117798
1,0.098267,0.123291,0.089722,0.103760,0.100708,0.119019,0.099487,0.121460,0.097656,0.113525,...,0.114136,0.129395,0.106201,0.126953,0.089722,0.102539,0.095215,0.112305,0.121460,0.140381
2,0.092163,0.104980,0.094604,0.108032,0.093384,0.109863,0.104980,0.124512,0.097656,0.109863,...,0.107422,0.131836,0.088501,0.098877,0.098267,0.116577,0.101929,0.120239,0.122070,0.142822
3,0.095215,0.111084,0.103149,0.120850,0.091553,0.112305,0.108032,0.123901,0.094604,0.109863,...,0.102539,0.127563,0.112305,0.134277,0.095215,0.106812,0.098877,0.118408,0.119019,0.139160
4,0.083008,0.092163,0.084229,0.100098,0.088501,0.105591,0.103149,0.111694,0.109863,0.128784,...,0.093994,0.111694,0.109253,0.122681,0.097656,0.114136,0.098877,0.120239,0.114136,0.132446


# Multi-Channel Signal Processing

Decompose audio signal into multiple wavelet sub-bands using Wavelet Packet Transform (WPT)

Complete the process with a sample case before generalizing the framework.

In [6]:
t1 = audio_features["AE_table_case001"] # Table data from case 1
s1 = audio_features["AE_spindle_case001"] # Spindle data from case 1

In [8]:
t1

0       0.087280
1       0.098267
2       0.092163
3       0.095215
4       0.083008
          ...   
8995    0.080566
8996    0.085449
8997    0.101318
8998    0.127563
8999    0.123291
Name: AE_table_case001, Length: 9000, dtype: float64

In [10]:
import plotly.express as px

In [19]:
fig = px.line(x=t1.index, y=t1, title='Acoustic Emissions at the Table')

fig.update_layout(
    xaxis_title="Sample",
    yaxis_title="Acoustic Emissions" # What units are these?
)

fig.show()

Perform WPT

In [23]:
wp_t1 = pywt.WaveletPacket(data=t1, wavelet='db1', mode='symmetric')
wp_s1 = pywt.WaveletPacket(data=s1, wavelet='db1', mode='symmetric')

HOW DO I ANALYZE THE DATA?